In [77]:
import os
import sys
sys.path.append('/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/')
import argparse
import numpy as np
import nibabel as nib
import scipy.io as sio
import subprocess
from scipy.stats import zscore
from nibabel.nicom import dicomreaders
import pydicom as dicom  # type: ignore
import time
from glob import glob
import shutil
from nilearn.image import new_img_like
import joblib
import rtCommon.utils as utils
from rtCommon.utils import loadConfigFile
import pickle5 as pickle
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
# from rtCommon.fileClient import FileInterface
# import rtCommon.projectUtils as projUtils
# from rtCommon.imageHandling import readRetryDicomFromFileInterface, getDicomFileName, convertDicomImgToNifti


argParser = argparse.ArgumentParser()
argParser.add_argument('--config', '-c', default='sub002.ses3.toml', type=str, help='experiment file (.json or .toml)')
argParser.add_argument('--skipPre', '-s', default=0, type=int, help='skip preprocess or not')
argParser.add_argument('--skipGreedy', '-g', default=0, type=int, help='skip greedy or not')
argParser.add_argument('--testRun', '-t', default=None, type=int, help='testRun, can be [None,1,2,3,4,5,6,7,8]')
argParser.add_argument('--scan_asTemplate', '-a', default=1, type=int, help="which scan's middle dicom as Template?")

args = argParser.parse_args("")
from rtCommon.cfg_loading import mkdir,cfg_loading
# config="sub001.ses2.toml"
cfg = cfg_loading(args.config)
os.chdir("/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/")
cfg.session

toml filename=/gpfs/milgram/project/turk-browne/projects/rtSynth_rt/projects/rtSynth_rt/conf/sub002.ses3.toml


3

In [88]:
import pandas as pd
runRecording = pd.read_csv(f"{cfg.recognition_dir}../runRecording.csv")
actualFeedbackScans = list(runRecording['run'].iloc[list(np.where(1==1*(runRecording['type']=='feedback'))[0])]) # can be [1,2,3,4,5,6,7,8] or [1,2,4,5]


def getSuc(t,curr_run,cfg,x):
    if cfg.session==2:
        thresholds={1:0.6,2:0.6,3:0.6,4:0.6,5:0.6,6:0.6,7:0.65,8:0.65,9:0.65,10:0.70}
        headLengths={1:8,2:5,3:5,4:5,5:5,6:5,7:5,8:5,9:5,10:5}
    elif cfg.session==3:
        thresholds={1:0.7,2:0.65,3:0.60,4:0.65,5:0.70,6:0.70,7:0.70,8:0.70,9:0.75,10:0.80,11:0.80,12:0.80}
        headLengths={1:7,2:5,3:5,4:5,5:5,6:5,7:5,8:5,9:6,10:5,11:5,12:x}

    threshold=thresholds[curr_run]
    successfulTrial=0
    perfectTrial=0
    reward1=0
    reward5=0
    reward9=0
    reward13=0
    
    headLength = headLengths[curr_run]
    curr_run
    headLength=6 #len(t)-12*14
    for currTrial in range(12):
        startID = currTrial*(6+3+5)+headLength
        endID = startID + 5
        trial_data=t[startID:endID]
        # print(trial_data)
        if np.sum(trial_data >= threshold) > 0:
            successfulTrial+=1
        if np.sum(trial_data >= threshold) >= 3:
            perfectTrial+=1
        if np.sum(trial_data >= threshold) >= 3:
            reward1+=1
        if np.sum(trial_data >= threshold) == 2:
            reward5+=1
        if np.sum(trial_data >= threshold) == 1:
            reward9+=1
        if np.sum(trial_data >= threshold) == 0:
            reward13+=1
    print(f"successfulTrial={successfulTrial}")
    print(f"perfectTrial={perfectTrial}")
    print(f"1:{reward1} 5:{reward5} 9:{reward9} 13:{reward13}")
    
for curr_run,curr_scan in enumerate(actualFeedbackScans):
    t = np.load(f"{cfg.feedback_dir}B_probs_{curr_scan}.npy")
    # print(len(t))
    print(f"run={curr_run+1} scan={curr_scan}", end=' ')
    getSuc(t,curr_run+1,cfg)

run=1 scan=3 successfulTrial=9
perfectTrial=5
1:5 5:3 9:1 13:3
run=2 scan=4 successfulTrial=11
perfectTrial=4
1:4 5:4 9:3 13:1
run=3 scan=5 successfulTrial=10
perfectTrial=2
1:2 5:4 9:4 13:2
run=4 scan=6 successfulTrial=9
perfectTrial=0
1:0 5:3 9:6 13:3
run=5 scan=7 successfulTrial=7
perfectTrial=1
1:1 5:1 9:5 13:5
run=6 scan=8 successfulTrial=5
perfectTrial=0
1:0 5:2 9:3 13:7
run=7 scan=9 successfulTrial=9
perfectTrial=2
1:2 5:3 9:4 13:3
run=8 scan=10 successfulTrial=6
perfectTrial=3
1:3 5:2 9:1 13:6
run=9 scan=11 successfulTrial=8
perfectTrial=0
1:0 5:2 9:6 13:4
run=10 scan=12 successfulTrial=5
perfectTrial=0
1:0 5:0 9:5 13:7
run=11 scan=13 successfulTrial=6
perfectTrial=2
1:2 5:2 9:2 13:6
run=12 scan=14 successfulTrial=10
perfectTrial=2
1:2 5:3 9:5 13:2


In [90]:

def getSuc(t,curr_run,cfg,x):
    if cfg.session==2:
        thresholds={1:0.6,2:0.6,3:0.6,4:0.6,5:0.6,6:0.6,7:0.65,8:0.65,9:0.65,10:0.70}
        headLengths={1:8,2:5,3:5,4:5,5:5,6:5,7:5,8:5,9:5,10:5}
    elif cfg.session==3:
        thresholds={1:0.7,2:0.65,3:0.60,4:0.65,5:0.70,6:0.70,7:0.70,8:0.70,9:0.75,10:0.80,11:0.80,12:0.80}
        headLengths={1:7,2:5,3:5,4:5,5:5,6:5,7:5,8:5,9:6,10:5,11:5,12:x}

    threshold=thresholds[curr_run]
    successfulTrial=0
    perfectTrial=0
    reward1=0
    reward5=0
    reward9=0
    reward13=0
    
    headLength = headLengths[curr_run]
    curr_run
    headLength=6 #len(t)-12*14
    for currTrial in range(12):
        startID = currTrial*(6+3+5)+headLength
        endID = startID + 5
        trial_data=t[startID:endID]
        # print(trial_data)
        if np.sum(trial_data >= threshold) > 0:
            successfulTrial+=1
        if np.sum(trial_data >= threshold) >= 3:
            perfectTrial+=1
        if np.sum(trial_data >= threshold) >= 3:
            reward1+=1
        if np.sum(trial_data >= threshold) == 2:
            reward5+=1
        if np.sum(trial_data >= threshold) == 1:
            reward9+=1
        if np.sum(trial_data >= threshold) == 0:
            reward13+=1
    print(f"successfulTrial={successfulTrial}")
    print(f"perfectTrial={perfectTrial}")
    print(f"1:{reward1} 5:{reward5} 9:{reward9} 13:{reward13}")
    
for x in [1,2,3,4,5,6,7,8,9,10]:
    getSuc(t,curr_run+1,cfg,x)

successfulTrial=10
perfectTrial=2
1:2 5:3 9:5 13:2
successfulTrial=10
perfectTrial=2
1:2 5:3 9:5 13:2
successfulTrial=10
perfectTrial=2
1:2 5:3 9:5 13:2
successfulTrial=10
perfectTrial=2
1:2 5:3 9:5 13:2
successfulTrial=10
perfectTrial=2
1:2 5:3 9:5 13:2
successfulTrial=10
perfectTrial=2
1:2 5:3 9:5 13:2
successfulTrial=10
perfectTrial=2
1:2 5:3 9:5 13:2
successfulTrial=10
perfectTrial=2
1:2 5:3 9:5 13:2
successfulTrial=10
perfectTrial=2
1:2 5:3 9:5 13:2
successfulTrial=10
perfectTrial=2
1:2 5:3 9:5 13:2


In [91]:
curr_run+1

12